# Feature Engineering and Modelling

## <u>The Physics & Economics of Price: Why Temperature is Non-Linear</u>

`temp_c` is arguably the most critical external factor for predicting `price_actual`. However, our correlation matrix showed a low linear correlation ($\approx 0.11$). This suggests the relationship is **non-linear**, and using a standard linear model without transformation would result in significant under-fitting.

**<u>The Core Axiom: The Causal Chain</u>**

To understand why the data looks this way, we must look at the fundamental chain of causality in energy markets:

> **Weather influences Human Behavior $\rightarrow$ Human Behavior drives Demand $\rightarrow$ Demand drives Price.**

The grid does not care about temperature; it cares about how **humans react** to temperature. This reaction is not uniform, it depends entirely on the season.

**<u>Analysis of the "U-Shaped" Relationship</u>**

We can categorize this human behavior into three distinct thermal regimes:

1.  **The Summer Regime (Cooling Load):**
    During hot weather ($> 25^\circ C$), human behavior shifts towards seeking comfort. People turn on Air Conditioning. This creates a massive surge in **Demand**, which instantly drives **Price** up.
    *   *Mathematical Behavior:* Positive Linear Relationship ($Slope > 0$).

2.  **The Winter Regime (Heating Load):**
    During freezing weather ($< 5^\circ C$), human behavior shifts towards survival. People maximize electric heating systems. This spike in **Demand** forces expensive peaker plants online, driving **Price** up.
    *   *Mathematical Behavior:* Inverse Linear Relationship ($Slope < 0$).

3.  **The "Comfort Zone" (Shoulder Months):**
    During mild weather ($15^\circ C - 20^\circ C$), human behavior becomes passive. Windows are opened, and neither AC nor Heating is required. **Demand** collapses to its minimum baseload, resulting in the lowest **Prices** of the year.
    *   *Mathematical Behavior:* The Global Minimum of the curve.

**<u>The Solution: Polynomial Transformation</u>**

Since the relationship goes "Down" (Winter), "Flattens" (Spring), and then "Up" (Summer), it mathematically forms a **Parabola** ($y = x^2$).

To capture this, we cannot simply use `temp_c`. We must apply a **Polynomial Transformation**. This achieves two goals:
1.  **Curvature:** By introducing `temp_squared`, the model can fit the "U-Shape" described above.
2.  **Interaction:** By using `PolynomialFeatures`, we also capture complex weather interactions (e.g., `temp_c * solar_radiation`), allowing the model to understand scenarios like *"It is hot (High Demand), but the Sun is shining (High Supply)."*

In [ ]:
# Imports and Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# Preprocessing and Pipelines
from sklearn.preprocessing import RobustScaler, PolynomialFeatures # RobustScaler is more robust to outliers than other scalers.
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Models 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor

# Validation and Metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit

In [2]:
df = pd.read_csv("../data/processed/trained_data.csv")
display(df.head())

,datetime_beginning_ept,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
0,2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
1,2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2,2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
3,2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
4,2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [3]:
# Converting the datetime column to datetime type
df["datetime_beginning_ept"] = pd.to_datetime(df["datetime_beginning_ept"])

# Setting back the datetime index
df = df.set_index("datetime_beginning_ept")
df.sort_index(inplace=True)
display(df.head())

,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
datetime_beginning_ept,,,,,,,,,,
2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [4]:
# Verifying the index
df.index

DatetimeIndex(['2024-01-02 00:00:00', '2024-01-02 01:00:00',
               '2024-01-02 02:00:00', '2024-01-02 03:00:00',
               '2024-01-02 04:00:00', '2024-01-02 05:00:00',
               '2024-01-02 06:00:00', '2024-01-02 07:00:00',
               '2024-01-02 08:00:00', '2024-01-02 09:00:00',
               ...
               '2024-12-30 15:00:00', '2024-12-30 16:00:00',
               '2024-12-30 17:00:00', '2024-12-30 18:00:00',
               '2024-12-30 19:00:00', '2024-12-30 20:00:00',
               '2024-12-30 21:00:00', '2024-12-30 22:00:00',
               '2024-12-30 23:00:00', '2024-12-31 00:00:00'],
              dtype='datetime64[ns]', name='datetime_beginning_ept', length=8736, freq=None)

In [5]:
# Summary Function 
def summarize_df(df, df_name="df"):
    """
    Display key information about a DataFrame:
    - info()
    - describe()
    - duplicated rows
    - count of missing values
    """
    print(f"===== DataFrame ({df_name.upper()}) Summary =====")
    print("===== DataFrame Index =====")
    display(df.index)
    print("===== DataFrame Info =====")
    df.info()
    print("\n===== DataFrame Description =====")
    display(df.describe(include='all'))  # include='all' to describe non-numeric columns too
    print("\n===== Duplicate Rows =====")
    duplicates = df[df.duplicated(keep=False)]
    if not duplicates.empty:
        display(duplicates)
    else:
        print("No duplicate rows found.")
    print("\n===== Missing Values per Column =====")
    print(df.isna().sum())


summarize_df(df, "train_df")

===== DataFrame (TRAIN_DF) Summary =====
===== DataFrame Index =====


DatetimeIndex(['2024-01-02 00:00:00', '2024-01-02 01:00:00',
               '2024-01-02 02:00:00', '2024-01-02 03:00:00',
               '2024-01-02 04:00:00', '2024-01-02 05:00:00',
               '2024-01-02 06:00:00', '2024-01-02 07:00:00',
               '2024-01-02 08:00:00', '2024-01-02 09:00:00',
               ...
               '2024-12-30 15:00:00', '2024-12-30 16:00:00',
               '2024-12-30 17:00:00', '2024-12-30 18:00:00',
               '2024-12-30 19:00:00', '2024-12-30 20:00:00',
               '2024-12-30 21:00:00', '2024-12-30 22:00:00',
               '2024-12-30 23:00:00', '2024-12-31 00:00:00'],
              dtype='datetime64[ns]', name='datetime_beginning_ept', length=8736, freq=None)

===== DataFrame Info =====
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8736 entries, 2024-01-02 00:00:00 to 2024-12-31 00:00:00
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price_actual        8736 non-null   float64
 1   hour_of_day         8736 non-null   float64
 2   day_of_week         8736 non-null   float64
 3   month               8736 non-null   float64
 4   price_1h_ago        8736 non-null   float64
 5   price_24h_ago       8736 non-null   float64
 6   avg_price_last_24h  8736 non-null   float64
 7   temp_c              8736 non-null   float64
 8   wind_kph            8736 non-null   float64
 9   solar_radiation     8736 non-null   float64
dtypes: float64(10)
memory usage: 750.8 KB

===== DataFrame Description =====


,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
count,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000,8736.000000
mean,33.278590,11.499771,3.000229,6.514766,33.280508,33.313805,33.300629,13.217903,9.870650,173.569254
std,28.297715,6.922930,1.999886,3.437097,28.297486,28.294552,14.345797,9.908276,5.366993,250.165110
min,-34.773600,0.000000,0.000000,1.000000,-34.773600,-34.773600,11.098400,-11.400000,0.200000,0.000000
25%,19.222125,5.750000,1.000000,4.000000,19.222125,19.270475,24.265900,5.200000,6.100000,0.000000
50%,26.266850,11.500000,3.000000,7.000000,26.266850,26.314500,30.366700,13.650000,8.700000,8.000000
75%,37.270625,17.250000,5.000000,10.000000,37.283700,37.302700,37.931675,21.000000,12.600000,307.000000
max,492.583300,23.000000,6.000000,12.000000,492.583300,492.583300,117.905700,37.000000,36.800000,980.000000



===== Duplicate Rows =====
No duplicate rows found.

===== Missing Values per Column =====
price_actual          0
hour_of_day           0
day_of_week           0
month                 0
price_1h_ago          0
price_24h_ago         0
avg_price_last_24h    0
temp_c                0
wind_kph              0
solar_radiation       0
dtype: int64


## Training and Evaluating Multiple Regression Models

In [6]:
# Since we are using ColumnTransformer, we need to have the columns' names available
X = df.drop(columns=["price_actual"])
y = df["price_actual"]

In [7]:
display(df.head())

,price_actual,hour_of_day,day_of_week,month,price_1h_ago,price_24h_ago,avg_price_last_24h,temp_c,wind_kph,solar_radiation
datetime_beginning_ept,,,,,,,,,,
2024-01-02 00:00:00,21.3249,0.0,2.0,1.0,23.3484,31.3827,29.3789,2.0,10.8,0.0
2024-01-02 01:00:00,19.6885,1.0,2.0,1.0,21.3249,20.0838,28.9598,1.2,8.0,0.0
2024-01-02 02:00:00,20.0916,2.0,2.0,1.0,19.6885,17.6052,28.9433,0.4,6.4,0.0
2024-01-02 03:00:00,18.6212,3.0,2.0,1.0,20.0916,19.7673,29.0469,0.5,10.7,0.0
2024-01-02 04:00:00,18.6391,4.0,2.0,1.0,18.6212,17.0687,28.9992,-0.3,12.0,0.0


In [ ]:
weather_cols: list[str] = ["temp_c", "wind_kph", "solar_radiation"]
other_cols: list[str] = ["price_1h_ago", "price_24h_ago", "avg_price_last_24h", "hour_of_day", "day_of_week", "month"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42, shuffle=False)


# Used for Time Series based Cross-Validation 
tscv = TimeSeriesSplit(n_splits=60, test_size=24) # Forecast horizon of 24 hours over a period of 60 days (1440 hours)

"""Pipeline for weather columns"""
weather_poly_scaled = Pipeline(steps=[
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", RobustScaler())
])

"""Pipeline for other columns"""
other_scaled = Pipeline(steps=[
    ("scaler", RobustScaler())
])

"""Processor for Linear and Non-Distance model"""
processor_scaled = ColumnTransformer(
    transformers=[
        ("weather_columns", weather_poly_scaled, weather_cols),
        ("other_columns", other_scaled, other_cols)
    ],
    remainder="drop"
)

"""Processor for Tree-model"""
processor_trees = ColumnTransformer(
    transformers=[
        ("all_columns", "passthrough", weather_cols + other_cols)
    ],
    remainder="drop"
)

# Dict of all pipelines
pipelines: dict = {
    "LinearRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("lr", LinearRegression())
    ]),

    "RidgeRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("ridge", Ridge())
    ]),

    "LassoRegression": Pipeline(steps=[
        ("process", processor_scaled),
        ("lasso", Lasso())
    ]),

    "KNN": Pipeline(steps=[
        ("process", processor_scaled),
        ("knn", KNeighborsRegressor())
    ]),

    "BaggingRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("br", BaggingRegressor(random_state=42, verbose=1))
    ]),

    "RandomForestRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("rfr", RandomForestRegressor(random_state=42, verbose=1))
    ]),

    "GradientBoostingRegressor": Pipeline(steps=[
        ("process", processor_trees),
        ("gbr", GradientBoostingRegressor(random_state=42, verbose=1))
    ])
}

# Hyperparameter grids for each model
param_grid: dict = {
    "Ridge": {"alpha": [0.01, 0.1, 1.0, 10.0]},
    "Lasso": {"alpha": [0.001, 0.01, 0.1, 1.0]},
    "KNN": {
        "n_neighbors": [3, 5, 7, 9],
        "weights": ["uniform", "distance"]
    },
    "RandomForestRegressor": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "max_features": ["sqrt", "log2"]
    },
    "BaggingRegressor": {
        "n_estimators": [10, 50, 100], # Uses Decision Tree as base estimator by default
        "max_samples": [0.6, 0.8, 1.0]
    },
    "GradientBoostingRegressor": {
        "n_estimators": [100, 200],
        "learning_rate": [0.05, 0.1],
        "max_depth": [2, 3, 4]
    },
}

# CROSS VALIDATION SETUP
results: list = []  #Stores the CV_Scores for each model
estimators: dict = {}
cv_score: float = 0.0
best_score: float = 0.0

print(f"Starting Training loop on {len(pipelines)} models...")
print("-" * 50)

for name, pipeline in pipelines.items():
    print(f"Training (Fitting) {name}")

    # Pipeline uses it to go through all parameters
    # pipelien__parameter: values
    search_params: dict = {} 

    step_name: str = "" # To prevent UnboundError

    # Identify which algorithm to it is 
    if "RidgeRegression" in name: grid_key, step_name = "Ridge", "ridge"
    elif "LassoRegression" in name: grid_key, step_name = "Lasso", "lasso"
    elif "KNN" in name: grid_key, step_name = "KNN", "knn"
    elif "RandomForestRegressor" in name: grid_key, step_name = "RandomForestRegressor", "rfr"
    elif "BaggingRegressor" in name: grid_key, step_name = "BaggingRegressor", "br"
    elif "GradientBoostingRegressor" in name: grid_key, step_name = "GradientBoostingRegressor", "gbr"
    else: grid_key = None  # No hyperparameter tuning for Linear Regression


    # Build the parameter grid for GridSearchCV
    if grid_key and grid_key in param_grid:
        for param, values in param_grid[grid_key].items():
            search_params[f"{step_name}__{param}"] = values

    # Run GridSearchCV if the model is in the param_grid
    if search_params: 
        model = GridSearchCV(
            pipeline, 
            param_grid=search_params, 
            cv=tscv, 
            scoring="neg_root_mean_squared_error", 
            n_jobs=-1, 
            verbose=1
        )

        # Fitting the model on the entire training data
        model.fit(X_train, y_train)
        final_model = model.best_estimator_ # Best pipeline with best hyperparameters
        best_score = round(-model.best_score_, 4) # Best CV score (lowest RMSE)
        best_params = model.best_params_ # Best hyperparameters

        # Saving the CV results for analysis
        cv_results = pd.DataFrame(model.cv_results_)
        display(cv_results.sort_values("mean_test_score"))
        display(cv_results[["params","mean_test_score","std_test_score"]].sort_values("mean_test_score"))


    else: # If the model was not in the param_grid, fit the pipeline directly
        cv_scores = cross_val_score(
            pipeline,
            X_train, 
            y_train, 
            cv=tscv, 
            scoring="neg_root_mean_squared_error"
         )

        cv_score = round(-cv_scores.mean(), 4) # AVG_RMSE 
        best_params = "Default"

        # Fitting the model on the entire training data
        pipeline.fit(X_train, y_train)
        final_model = pipeline

    
    # Store results
    results.append({
        "Model": name,
        "CV_RMSE": best_score if search_params else cv_score,
        "Best Params": best_params
    })

    # Saving the model object
    estimators[name] = final_model 
    print(f"Completed training for {name}. CV_RMSE: {results[-1]["CV_RMSE"]:.4f}")


# LEADERBOARD
print("-" * 50)
print("Printing the Model Leaderboard sorted by CV_RMSE:")
leaderb_df = pd.DataFrame(results).sort_values("CV_RMSE", ascending=True)
display(leaderb_df)

# SAVING MODELS 
print("Saving models to disk...")
joblib.dump(estimators, "../src/estimators.pkl") 
print("Done.")

Starting Training loop on 7 models...
--------------------------------------------------
Training (Fitting) LinearRegression
Completed training for LinearRegression. CV_RMSE: 16.7235
Training (Fitting) RidgeRegression
Fitting 60 folds for each of 4 candidates, totalling 240 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_ridge__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split53_test_score,split54_test_score,split55_test_score,split56_test_score,split57_test_score,split58_test_score,split59_test_score,mean_test_score,std_test_score,rank_test_score
0,0.060409,0.017142,0.018031,0.011627,0.01,{'ridge__alpha': 0.01},-2.966584,-4.676730,-9.162159,-4.005375,...,-5.509084,-8.945399,-30.786604,-37.311925,-40.243493,-25.770434,-36.473022,-16.723527,11.579799,4
1,0.064507,0.018625,0.020721,0.013724,0.10,{'ridge__alpha': 0.1},-2.966220,-4.676217,-9.162261,-4.004919,...,-5.508878,-8.946384,-30.786942,-37.312410,-40.243898,-25.770690,-36.472903,-16.723396,11.580040,3
2,0.070101,0.022453,0.020823,0.013889,1.00,{'ridge__alpha': 1.0},-2.962617,-4.671102,-9.163227,-4.000496,...,-5.506847,-8.956096,-30.790264,-37.317193,-40.247895,-25.773216,-36.471731,-16.722114,11.582413,2
3,0.064014,0.023742,0.018839,0.011072,10.00,{'ridge__alpha': 10.0},-2.929523,-4.621486,-9.168958,-3.966624,...,-5.489208,-9.041293,-30.818927,-37.359318,-40.283687,-25.794928,-36.462000,-16.711425,11.602771,1


,params,mean_test_score,std_test_score
0,{'ridge__alpha': 0.01},-16.723527,11.579799
1,{'ridge__alpha': 0.1},-16.723396,11.580040
2,{'ridge__alpha': 1.0},-16.722114,11.582413
3,{'ridge__alpha': 10.0},-16.711425,11.602771


Completed training for RidgeRegression. CV_RMSE: 16.7114
Training (Fitting) LassoRegression
Fitting 60 folds for each of 4 candidates, totalling 240 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_lasso__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split53_test_score,split54_test_score,split55_test_score,split56_test_score,split57_test_score,split58_test_score,split59_test_score,mean_test_score,std_test_score,rank_test_score
3,0.061457,0.015624,0.016936,0.009990,1.000,{'lasso__alpha': 1.0},-5.767444,-4.882118,-9.728694,-5.238728,...,-5.366948,-10.031750,-30.951034,-38.121383,-40.577025,-25.646150,-36.308846,-16.985666,11.512463,4
0,0.103983,0.023618,0.021011,0.012837,0.001,{'lasso__alpha': 0.001},-2.963530,-4.669060,-9.159725,-4.000274,...,-5.505533,-8.946875,-30.789488,-37.315228,-40.248267,-25.770209,-36.471326,-16.721961,11.582160,3
1,0.078709,0.013190,0.020657,0.011244,0.010,{'lasso__alpha': 0.01},-2.935203,-4.599951,-9.138943,-3.950237,...,-5.474834,-8.964108,-30.816666,-37.346465,-40.291349,-25.771440,-36.458430,-16.707359,11.602985,2
2,0.069885,0.018450,0.019873,0.012557,0.100,{'lasso__alpha': 0.1},-2.739097,-4.133708,-9.028115,-3.658898,...,-5.260677,-9.196576,-30.856528,-37.499363,-40.385222,-25.781598,-36.421182,-16.609558,11.703908,1


...

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gbr__learning_rate,param_gbr__max_depth,param_gbr__n_estimators,params,split0_test_score,split1_test_score,...,split53_test_score,split54_test_score,split55_test_score,split56_test_score,split57_test_score,split58_test_score,split59_test_score,mean_test_score,std_test_score,rank_test_score
11,6.562794,0.561922,0.005874,0.004105,0.10,4,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-3.116595,-3.230457,...,-7.032744,-8.665663,-24.926106,-38.271667,-34.805285,-20.008026,-35.560377,-16.968918,11.923645,12
9,5.132361,0.484334,0.005461,0.005019,0.10,3,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-2.943074,-2.998947,...,-6.485563,-7.435284,-25.710513,-41.875279,-36.975384,-23.381570,-36.842715,-16.742305,12.304394,11
10,3.215228,0.283737,0.005485,0.004027,0.10,4,100,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-2.941354,-3.220420,...,-6.109535,-8.257078,-25.130933,-38.379613,-35.730191,-22.945161,-35.830970,-16.704648,11.939604,10
8,2.595214,0.317456,0.007014,0.007213,0.10,3,100,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-2.932137,-3.210158,...,-6.272809,-7.461097,-26.079325,-40.448112,-37.312771,-24.926177,-37.574109,-16.481645,12.085277,9
5,6.486839,0.606241,0.005270,0.002487,0.05,4,200,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-3.165008,-3.153383,...,-6.331944,-7.803013,-25.320839,-39.169866,-36.002606,-24.751289,-37.142943,-16.467706,11.853726,8
3,5.147134,0.443179,0.006567,0.006327,0.05,3,200,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-3.044464,-3.223858,...,-6.170956,-7.516750,-25.952124,-38.433211,-36.985932,-24.584535,-37.917377,-16.463657,11.946017,7
0,1.730357,0.208847,0.005245,0.002825,0.05,2,100,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-3.604988,-3.997532,...,-5.832822,-7.953272,-27.759677,-36.086347,-38.724130,-25.419993,-37.154577,-16.447109,11.600826,6
7,3.456206,0.361918,0.007194,0.007887,0.10,2,200,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-2.789366,-2.926314,...,-6.371574,-7.460525,-26.602610,-36.697482,-37.579030,-24.667135,-37.245793,-16.436475,12.089687,5
4,3.260994,0.283280,0.005602,0.006538,0.05,4,100,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-3.391818,-3.476829,...,-6.140852,-8.129032,-25.923066,-37.670757,-36.518643,-25.259401,-38.793438,-16.355667,11.780389,4
1,3.590240,0.616111,0.005332,0.004680,0.05,2,200,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-3.082799,-3.289567,...,-6.028683,-7.521451,-27.127453,-35.910516,-38.468243,-25.125393,-37.276636,-16.343469,11.754960,3


,params,mean_test_score,std_test_score
11,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.968918,11.923645
9,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.742305,12.304394
10,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.704648,11.939604
8,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.481645,12.085277
5,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.467706,11.853726
3,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.463657,11.946017
0,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.447109,11.600826
7,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ...",-16.436475,12.089687
4,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.355667,11.780389
1,"{'gbr__learning_rate': 0.05, 'gbr__max_depth':...",-16.343469,11.754960


Completed training for GradientBoostingRegressor. CV_RMSE: 16.3008
--------------------------------------------------
Printing the Model Leaderboard sorted by CV_RMSE:


,Model,CV_RMSE,Best Params
5,RandomForestRegressor,15.9200,"{'rfr__max_depth': 10, 'rfr__max_features': 's..."
6,GradientBoostingRegressor,16.3008,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ..."
4,BaggingRegressor,16.4381,"{'br__max_samples': 0.6, 'br__n_estimators': 100}"
2,LassoRegression,16.6096,{'lasso__alpha': 0.1}
1,RidgeRegression,16.7114,{'ridge__alpha': 10.0}
0,LinearRegression,16.7235,Default
3,KNN,17.2224,"{'knn__n_neighbors': 9, 'knn__weights': 'dista..."


Saving models to disk...
Done.


## Predicting and Calculating Evaluation Metrics (RMSE and $R^2$)

In [10]:
from sklearn.metrics import root_mean_squared_error as rmse, r2_score
import pandas as pd

# Retrieving estimators dictionary
print("Loading models from disk...")
estimators = joblib.load("../src/estimators.pkl")

# Models
rf_model = estimators["RandomForestRegressor"]
gbr_model = estimators["GradientBoostingRegressor"]

# CV RMSE from leaderboard
rf_cv_rmse = leaderb_df.loc[
    leaderb_df["Model"] == "RandomForestRegressor", "CV_RMSE"
].values[0]

gbr_cv_rmse = leaderb_df.loc[
    leaderb_df["Model"] == "GradientBoostingRegressor", "CV_RMSE"
].values[0]

# Predictions
rf_pred = rf_model.predict(X_test)
gbr_pred = gbr_model.predict(X_test)

# Test metrics
rf_rmse = round(rmse(y_test, rf_pred), 4)
gbr_rmse = round(rmse(y_test, gbr_pred), 4)

rf_r2 = round(r2_score(y_test, rf_pred), 4)
gbr_r2 = round(r2_score(y_test, gbr_pred), 4)

# Generalization gap (absolute difference)
rf_gap = round(abs(rf_rmse - rf_cv_rmse), 4)
gbr_gap = round(abs(gbr_rmse - gbr_cv_rmse), 4)

# Dynamic best vs comparison (lower RMSE is better)
if rf_rmse < gbr_rmse:
    rows = [
        ["RandomForestRegressor", "Best Model", rf_cv_rmse, rf_rmse, rf_r2, rf_gap],
        ["GradientBoostingRegressor", "Comparison Model", gbr_cv_rmse, gbr_rmse, gbr_r2, gbr_gap]
    ]
else:
    rows = [
        ["GradientBoostingRegressor", "Best Model", gbr_cv_rmse, gbr_rmse, gbr_r2, gbr_gap],
        ["RandomForestRegressor", "Comparison Model", rf_cv_rmse, rf_rmse, rf_r2, rf_gap]
    ]

# Final DataFrame
final_results = pd.DataFrame(
    rows,
    columns=["Model", "Type", "CV_RMSE", "Test_RMSE", "Test_R2", "Gap"]
)

display(leaderb_df)
display(final_results)

Loading models from disk...


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


,Model,CV_RMSE,Best Params
5,RandomForestRegressor,15.9200,"{'rfr__max_depth': 10, 'rfr__max_features': 's..."
6,GradientBoostingRegressor,16.3008,"{'gbr__learning_rate': 0.1, 'gbr__max_depth': ..."
4,BaggingRegressor,16.4381,"{'br__max_samples': 0.6, 'br__n_estimators': 100}"
2,LassoRegression,16.6096,{'lasso__alpha': 0.1}
1,RidgeRegression,16.7114,{'ridge__alpha': 10.0}
0,LinearRegression,16.7235,Default
3,KNN,17.2224,"{'knn__n_neighbors': 9, 'knn__weights': 'dista..."


,Model,Type,CV_RMSE,Test_RMSE,Test_R2,Gap
0,GradientBoostingRegressor,Best Model,16.3008,17.1221,0.4010,0.8213
1,RandomForestRegressor,Comparison Model,15.9200,17.8421,0.3495,1.9221


## PnL (Profit and Loss) Simulation and Evaluation Metrics

In [ ]:
# --- PnL SIMULATION: The "Battery Arbitrage" Strategy ---

# 1. Setup the Simulation Data
# We use the X_test/y_test data (The "Future" we haven't trained on)
best_model = estimators["GradientBoostingRegressor"]
df_sim = X_test.copy()
df_sim["price_actual"] = y_test
df_sim["price_predicted"] = best_model.predict(X_test)

# Add Date column for grouping
df_sim["date"] = df_sim.index.date

# 2. Define the Simulation Logic
def simulate_battery_revenue(daily_data):
    """
    Strategy: 
    - Buy at the hour with the LOWEST Price.
    - Sell at the hour with the HIGHEST Price.
    - Capacity: 100 MWh.
    """
    # ---------------------------------------------------------
    # SCENARIO A: Perfect Hindsight (The "Oracle")
    # We buy at the actual lowest and sell at the actual highest
    # ---------------------------------------------------------
    buy_price_perfect = daily_data["price_actual"].min()
    sell_price_perfect = daily_data["price_actual"].max()
    profit_perfect = (sell_price_perfect - buy_price_perfect) * 100 # 100 MWh
    
    # ---------------------------------------------------------
    # SCENARIO B: Our Model (The Gradient Boosting Regressor)
    # We make decisions based ONLY on "price_predicted"
    # ---------------------------------------------------------
    # Find the hour where our MODEL predicts the lowest/highest
    buy_hour_pred = daily_data["price_predicted"].idxmin()
    sell_hour_pred = daily_data["price_predicted"].idxmax()
    
    # Calculate profit using the ACTUAL prices at those specific hours
    # (You can't pay with predicted dollars, you pay actual market rates)
    buy_price_model = daily_data.loc[buy_hour_pred, "price_actual"]
    sell_price_model = daily_data.loc[sell_hour_pred, "price_actual"]
    
    profit_model = (sell_price_model - buy_price_model) * 100 # 100 MWh
    
    return pd.Series([profit_perfect, profit_model], index=["Perfect_Profit", "Model_Profit"])


# 3. Run Simulation Daily
daily_profits = df_sim.groupby("date").apply(simulate_battery_revenue)

print(f"No of rows of {len(daily_profits)}")
display(daily_profits.head())

# 4. Aggregate Results
total_perfect_profit = daily_profits['Perfect_Profit'].sum()
total_model_profit = daily_profits['Model_Profit'].sum()
efficiency = (total_model_profit / total_perfect_profit) * 100

print("-" * 50)
print("BATTERY ARBITRAGE SIMULATION RESULTS (Nov-Dec Test Period)")
print("-" * 50)
print(f"Total Potential Profit (Perfect Foresight): ${total_perfect_profit:,.2f}")
print(f"Total Realized Profit (Our Model):          ${total_model_profit:,.2f}")
print(f"Model Capture Efficiency:                   {efficiency:.1f}%")
print("-" * 50)

No of rows of 74


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_244\1070105645.py:48: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_profits = df_sim.groupby('date').apply(simulate_battery_revenue)


,Perfect_Profit,Model_Profit
date,,
2024-10-19,15485.76,2752.49
2024-10-20,4913.51,3327.24
2024-10-21,10120.01,5918.67
2024-10-22,11904.95,5230.39
2024-10-23,8910.42,7393.79


--------------------------------------------------
BATTERY ARBITRAGE SIMULATION RESULTS (Nov-Dec Test Period)
--------------------------------------------------
Total Potential Profit (Perfect Foresight): $428,227.85
Total Realized Profit (Our Model):          $202,434.38
Model Capture Efficiency:                   47.3%
--------------------------------------------------
